# Transaction.csv Analysis

In [0]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import warnings
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import gc
warnings.filterwarnings('ignore')

In [0]:
transaction_df = pd.read_csv('transaction.csv')
transaction_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN


In [0]:
transaction_df.shape

(590540, 17)

In [0]:
transaction_df.dtypes

TransactionID       int64
isFraud             int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
card1               int64
card2             float64
card3             float64
card4              object
card5             float64
card6              object
addr1             float64
addr2             float64
dist1             float64
dist2             float64
P_emaildomain      object
R_emaildomain      object
dtype: object

In [0]:
transaction_df.describe(include='all')

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540,590540.000000,581607.000000,588975.000000,588963,586281.000000,588969,524834.000000,524834.000000,238269.000000,37627.000000,496084,137291
unique,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,4,NaN,4,NaN,NaN,NaN,NaN,59,60
top,NaN,NaN,NaN,NaN,W,NaN,NaN,NaN,visa,NaN,debit,NaN,NaN,NaN,NaN,gmail.com,gmail.com
freq,NaN,NaN,NaN,NaN,439670,NaN,NaN,NaN,384767,NaN,439938,NaN,NaN,NaN,NaN,228355,57147
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,NaN,9898.734658,362.555488,153.194925,NaN,199.278897,NaN,290.733794,86.800630,118.502180,231.855423,NaN,NaN
std,1.704744e+05,0.183755,4.617224e+06,239.162522,NaN,4901.170153,157.793246,11.336444,NaN,41.244453,NaN,101.741072,2.690623,371.872026,529.053494,NaN,NaN
min,2.987000e+06,0.000000,8.640000e+04,0.251000,NaN,1000.000000,100.000000,100.000000,NaN,100.000000,NaN,100.000000,10.000000,0.000000,0.000000,NaN,NaN
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,NaN,6019.000000,214.000000,150.000000,NaN,166.000000,NaN,204.000000,87.000000,3.000000,7.000000,NaN,NaN
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,NaN,9678.000000,361.000000,150.000000,NaN,226.000000,NaN,299.000000,87.000000,8.000000,37.000000,NaN,NaN
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,NaN,14184.000000,512.000000,150.000000,NaN,226.000000,NaN,330.000000,87.000000,24.000000,206.000000,NaN,NaN


In [0]:
for column in transaction_df.columns:
    num_nans = transaction_df[column].isna().sum()
    print('percentage of NaNs for {}: {:.3f}'.format(
        column, 
        num_nans / len(transaction_df) * 100
    ))

percentage of NaNs for TransactionID: 0.000
percentage of NaNs for isFraud: 0.000
percentage of NaNs for TransactionDT: 0.000
percentage of NaNs for TransactionAmt: 0.000
percentage of NaNs for ProductCD: 0.000
percentage of NaNs for card1: 0.000
percentage of NaNs for card2: 1.513
percentage of NaNs for card3: 0.265
percentage of NaNs for card4: 0.267
percentage of NaNs for card5: 0.721
percentage of NaNs for card6: 0.266
percentage of NaNs for addr1: 11.126
percentage of NaNs for addr2: 11.126
percentage of NaNs for dist1: 59.652
percentage of NaNs for dist2: 93.628
percentage of NaNs for P_emaildomain: 15.995
percentage of NaNs for R_emaildomain: 76.752


TransactionID, TransactionDT, TransactionAmt, ProductCD, card1 has no missing values.

dist2 has high no. of NaNs 

In [0]:
def reduce_mem_usage(df, verbose=True):
    """
    Reduce dataframe size

    params:
    - df: dataframe to reduce the size of

    return:
    - dataframe of reduced size
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'] 
#                 'float128']
    start_mem = df.memory_usage().sum() / 1024**2    

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
#                 elif c_min > np.finfo(np.float128).min and c_max < np.finfo(np.float128).max:
#                     df[col] = df[col].astype(np.float128)
                    
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose: 
        print(
            'Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem
        ))

    return df

In [0]:
transaction_df = reduce_mem_usage(transaction_df)

Mem. usage decreased to 37.73 Mb (50.7% reduction)


# Data Preprocessing

In [0]:
# Label Encoding
# https://www.kaggle.com/kabure/extensive-eda-and-modeling-xgb-hyperopt
for f in transaction_df.drop('isFraud', axis=1).columns:
    if transaction_df[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(transaction_df[f].values))
        transaction_df[f] = lbl.transform(list(transaction_df[f].values)) 

In [0]:
y_df = transaction_df['isFraud']
x_df = transaction_df.drop(["isFraud", "dist2"], axis=1)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x_df, 
                                                    y_df, 
                                                    test_size=0.2, 
                                                    random_state=0)

In [0]:
lgb_params = {
        "objective" : "binary",
        "metric" : "AUC",
        "max_depth" : 2,
        "num_leaves" : 2,
        "learning_rate" : 0.055,
        "bagging_fraction" : 0.3,
        "feature_fraction" : 0.15,
        "lambda_l1" : 5,
        "lambda_l2" : 5,
        "verbosity" : 1
    }

In [0]:
lgb_train = lgb.Dataset(x_train, label=y_train)
model = lgb.train(lgb_params, lgb_train, 100)
y_pred = model.predict(x_test)

In [0]:
roc_auc_score(y_test, y_pred)

0.7448983361665535